# Reddit API
While there are many public datasets for Reddit, they are often years old or not updated freqently. The goal of the project is to provide information on Covid-19 and related legal advice, so we're dealing with rapidly developing news and would like to have access to recent activity, as well as historical Reddit posts in a specified period of time. The Reddit API (via PRAW as a Python Reddit API Wrapper, making implementation easier) is powerful and can provide a lot of data and metadata, however it is limited in scope for time periods and response size. To address that, we can use the Pushshift API, an API that provides access to a vast amount of public Reddit data that has been ammassed over the years and continues to get updated by the day (? pretty frequenly I'm pretty sure). Another option rather that is accessing the back-end Elasticsearch search engine endpoint (but doesn't work right now?).

Prototype 1: For the moment, Reddit data will use PRAW, which is relatively easy to retrieve. There is an Reddit Api call that can search within a subreddit, which utilizes a cloud-based search engine backend (like Apache Lucene) for internal site searches, but we'd be relying too much on Reddit's relevance algorithms. (There are other Reddit Api functions that get the top/new/hot/controversial posts in a subreddit, but they don't search queries. We could do this, but once again, the Reddit Api is limited in retrieval/time limit, so this would only be for special cases). 
Since to get historic data via Pushshift takes a while and we need to decide how to process a lot of data quickly, we won't use that for now.

In [1]:
import os
import time
import datetime
import praw
import requests
import datetime
import json
import random

## Using Pushshift API

Note:
Includes deleted posts...

In [2]:
# max batch size 1000
# time range field m doesn't work...?
# Retrieving top scored posts from the Coronavirus subreddit from 120 days ago
top = 'https://api.pushshift.io/reddit/search/submission/?subreddit=coronavirus&size=100&after=1575176400&score=>20&sort_type=score'

res = requests.get(top)
data_arr = res.json()['data']

In [3]:
data_arr
for k, obj in enumerate(data_arr):
    print(20*'-', k)
    print('Title:', obj['title'])
    print('Author:', obj['author'])
    print(obj['full_link'])
    print('Score:', obj['score'])
    epoch = obj['created_utc']
    print('Epoch:', epoch)
    print('Date:', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(epoch)))   

-------------------- 0
Title: Denmark shuts down all schools and universities to curb the spread of Corona
Author: archeeye
https://www.reddit.com/r/Coronavirus/comments/fh44cj/denmark_shuts_down_all_schools_and_universities/
Score: 21
Epoch: 1583959831
Date: 2020-03-11 15:50:31
-------------------- 1
Title: Denmark enters preliminary lockdown
Author: erta0015
https://www.reddit.com/r/Coronavirus/comments/fh3cem/denmark_enters_preliminary_lockdown/
Score: 21
Epoch: 1583956973
Date: 2020-03-11 15:02:53
-------------------- 2
Title: Florida school districts discussing virtual teaching strategies amid coronavirus outbreak
Author: Joe_Tazuna
https://www.reddit.com/r/Coronavirus/comments/fh6cey/florida_school_districts_discussing_virtual/
Score: 21
Epoch: 1583968234
Date: 2020-03-11 18:10:34
-------------------- 3
Title: Trump says he will address the nation from the Oval Office at 9 p.m. ET
Author: causeimnotdrunk
https://www.reddit.com/r/Coronavirus/comments/fh468t/trump_says_he_will_addr

In [4]:
print(len(data_arr))

100


## Using PRAW

Using Reddit's search algorithm from its listings (Reddit's data structure for the list of ranked posts.). It utilizes stemming.

In [5]:
# headers = {'User-agent': 'redditRetrival'}, needed to uniquely identify to prevent rate limiting
res = requests.get('https://www.reddit.com/r/all/search/.json?q=coronavirus&restrict_sr=1&limit=1000', headers = {'User-agent': 'redditRetrival'})
print(res)
vv = res.json()
vv

<Response [200]>


{'kind': 'Listing',
 'data': {'after': 't3_g6a1fr',
  'dist': 100,
  'facets': {},
  'modhash': '',
  'children': [{'kind': 't3',
    'data': {'approved_at_utc': None,
     'subreddit': 'askscience',
     'selftext': "This thread is for questions related to the current coronavirus outbreak.\n\n&gt;The Centers for Disease Control and Prevention (CDC) is closely monitoring developments around an outbreak of respiratory illness caused by a novel (new) coronavirus first identified in Wuhan, Hubei Province, China. Chinese authorities identified the new coronavirus, which has resulted in hundreds of confirmed cases in China, including cases outside Wuhan City, with additional cases being identified in a growing number of countries internationally. The first case in the United States was announced on January 21, 2020. There are ongoing investigations to learn more.\n\n- [CDC](https://www.cdc.gov/coronavirus/index.html)\n\n[China coronavirus: A visual guide - BBC News](https://www.bbc.com/news

# Using the 'public' Reddit API, aka through the normal search bar web interface, with .json extension. 

Advantage of this over PRAW is that there is no (rather insignificant) limit rate on requests, as well as request size limit. However, it can only pull 100, regardless of if it is search in subreddit or just the top/hot ranked in a subreddit. (But we probably won't need more than 100?)

https://www.reddit.com/r/all/search/.json?q=coronavirus&restrict_sr=1&limit=1000 \
https://www.reddit.com/r/Coronavirus/top/.json?t=all&limit=1000

# Adding some variables related to NY

In [6]:
il_counties = ['Illinois',
               ' IL ',
               '[IL]',
               '(IL)',
               '{IL}',
               'Adams',
               'Alexander',
               'Bond',
               'Boone',
               'Brown',
               'Bureau',
               'Calhoun',
               'Carroll',
               'Cass',
               'Champaign',
               'Christian',
               'Clark',
               'Clay',
               'Clinton',
               'Coles',
               'Cook',
               'Crawford',
               'Cumberland',
               'DeKalb',
               'DeWitt',
               'Douglas',
               'DuPage',
               'Edgar',
               'Edwards',
               'Effingham',
               'Fayette',
               'Ford',
               'Franklin',
               'Fulton',
               'Gallatin',
               'Greene',
               'Grundy',
               'Hamilton',
               'Hancock',
               'Hardin',
               'Henderson',
               'Henry',
               'Iroquois',
               'Jackson',
               'Jasper',
               'Jefferson',
               'Jersey',
               'Jo Daviess',
               'Johnson',
               'Kane',
               'Kankakee',
               'Kendall',
               'Knox',
               'Lake',
               'LaSalle',
               'Lawrence',
               'Lee',
               'Livingston',
               'Logan',
               'Macon',
               'Macoupin',
               'Madison',
               'Marion',
               'Marshall',
               'Mason',
               'Massac',
               'McDonough',
               'McHenry',
               'McLean',
               'Menard',
               'Mercer',
               'Monroe',
               'Montgomery',
               'Morgan',
               'Moultrie',
               'Ogle',
               'Peoria',
               'Perry',
               'Piatt',
               'Pike',
               'Pope',
               'Pulaski',
               'Putnam',
               'Randolph',
               'Richland',
               'Rock Island',
               'Saline',
               'Sangamon',
               'Schuyler',
               'Scott',
               'Shelby',
               'St. Clair',
               'Stark',
               'Stephenson',
               'Tazewell',
               'Union',
               'Vermillion',
               'Wabash',
               'Warren',
               'Wayne',
               'White',
               'Whiteside',
               'Will',
               'Williamson',
               'Winnebago',
               'Woodford'
]

# Cities and regions with close to or above Ithaca in population (Excluding Rome)
il_cities = ['Chicago',
             'Aurora',
             'Rockford',
             'Joliet',
             'Naperville',
             'Springfield',
             'Elgin',
             'Peoria',
             'Waukegan',
             'Champaign',
             'Bloomington',
             'Decatur',
             'Evanston',
             'Des Plaines',
             'Berwyn'
]

other_countries = ['Canada',
                   'Britain',
                   'Kingdom',
                   'Australia',
                   'Zealand',
                   'Germany',
                   'France',
                   'Spain',
                   'Italy',
                   ' UK ',
                   'U.K',
                   '[UK]',
                   '(UK)',
                   '{UK}'
]

other_states = ['Alabama',
                'Alaska',
                'Arizona',
                'Arkansas',
                'California',
                'Colorado',
                'Connecticut',
                'Delaware',
                'Florida',
                'Georgia',
                'Hawaii',
                'Idaho',
                'New York',
                'Indiana',
                'Iowa',
                'Kansas',
                'Kentucky',
                'Louisiana',
                'Maine',
                'Maryland',
                'Massachusetts',
                'Michigan',
                'Minnesota',
                'Mississippi',
                'Missouri',
                'Montana',
                'Nebraska',
                'Nevada',
                'New Hampshire',
                'New Jersey',
                'New Mexico',
                'North Carolina',
                'North Dakota',
                'Ohio',
                'Oklahoma',
                'Oregon',
                'Pennsylvania',
                'Rhode Island',
                'South Carolina',
                'South Dakota',
                'Tennessee',
                'Texas',
                'Utah',
                'Vermont',
                'Virginia',
                'Washington',
                'West Virginia',
                'Wisconsin',
                'Wyoming'
]

other_states_abbrev = [' AL '
                       ' AK ',
                       ' AZ ',
                       ' AR ',
                       ' CA ',
                       ' CO ',
                       ' CT ',
                       ' DE ', 
                       ' FL ',
                       ' GA ',
                       ' DC ',
                       ' HI ',
                       ' ID ',
                       ' NY ',
                       ' IN ',
                       ' IO ',
                       ' KS ',
                       ' KY ',
                       ' LA ',
                       ' ME ',
                       ' MD ',
                       ' MA ',
                       ' MI ',
                       ' MN ',
                       ' MS ',
                       ' MO ',
                       ' MT ',
                       ' NE ',
                       ' NV ',
                       ' NH ',
                       ' NJ ',
                       ' NM ',
                       ' NC ',
                       ' ND ',
                       ' OH ',
                       ' OK ',
                       ' OR ',
                       ' PA ',
                       ' RI ',
                       ' SC ',
                       ' SD ',
                       ' TN ',
                       ' TX ',
                       ' UT ',
                       ' VT ',
                       ' VA ',
                       ' WA ',
                       ' WV ',
                       ' WI ',
                       ' WY '
]

other_states_abbrev_1 = list(map(lambda x: '['+x[1:3:1]+']', other_states_abbrev))
other_states_abbrev_2 = list(map(lambda x: '('+x[1:3:1]+')', other_states_abbrev))
other_states_abbrev_3 = list(map(lambda x: '{'+x[1:3:1]+'}', other_states_abbrev))

# Top populous cities, excluding those in New York
other_cities = ['Los Angeles',
                'Houston',
                'New York City'
                'NYC',
                'Phoenix',
                'Philadelphia',
                'San Antonio',
                'San Diego',
                'Dallas',
                'San Jose',
                'Austin',
                'Jacksonville',
                'Fort Worth',
                'Columbus',
                'San Francisco',
                'Charlotte',
                'Indianapolis',
                'Seattle',
                'Denver',
                'Boston',
                'El Paso',
                'Detroit',
                'Nashville',
                'Portland',
                'Memphis',
                'Las Vegas',
                'Louisville',
                'Baltimore',
                'Milwaukee',
                'Albuquerque',
                'Tucson',
                'Fresno',
                'Mesa',
                'Sacramento',
                'Atlanta',
                'Miami',
                'Raleigh',
                'Omaha',
                'Oakland',
                'Minneapolis',
                'Tulsa',
                'Arlington',
                'Tampa',
                'New Orleans',
                'Wichita',
                'Cleveland',
                'Bakersfield',
                'Anaheim',
                'Honolulu',
                'Santa Ana',
                'Riverside',
                'Corpus Christi',
                'Lexington',
                'Stockton',
                'Henderson',
                'Saint Paul'
                'St. Louis',
                'Cincinnati',
                'Pittsburgh',
                'Greensboro',
                'Anchorage',
                'Plano',
                'Lincoln',
                'Orlando'
]

other_regions = ['Northeast',
                 'West Coast',
                 'South',
                 'Plains',
                 'Southwest'
]

il_counties = list(map(lambda x: x.lower(), il_counties))
il_cities = list(map(lambda x: x.lower(), il_cities))
other_countries = list(map(lambda x: x.lower(), other_countries))
other_states = list(map(lambda x: x.lower(), other_states))
other_states_abbrev = list(map(lambda x: x.lower(), other_states_abbrev))
other_states_abbrev_1 = list(map(lambda x: x.lower(), other_states_abbrev_1))
other_states_abbrev_2 = list(map(lambda x: x.lower(), other_states_abbrev_2))
other_states_abbrev_3 = list(map(lambda x: x.lower(), other_states_abbrev_3))
other_cities = list(map(lambda x: x.lower(), other_cities))
other_regions = list(map(lambda x: x.lower(), other_regions))

# Main Script to get All Reddit Data (testing above)

In [7]:
def get_submissions_from_pushshift(**kwargs):
    '''
    Use Pushshift API to pull in batches (max 1000 in a batch).
    
    Use parameters from:
    https://github.com/pushshift/api
    
    By default, results are returned based on created_utc in descending order, but if in ascending order
    if before/after is specified.  Must specify sort_order in that case if want time-window results descending.
    
    parameters:
        subreddit: subreddit to pull results
        after: Return results after this date
        before: Return results before this date
    
    returns:
        list of subreddit data
    '''
    headers = {'User-agent':'subm finder'}
    for i in range(50): # Try 50 times while still HTTP Error
        try:
            r = requests.get("https://api.pushshift.io/reddit/submission/search/", params=kwargs, headers=headers)
            data = r.json()
            break
        except:
            print(r)
#             headers['User-agent'] = headers['User-agent'] + str(i)
        time.sleep(random.uniform(2.0, 2.5))
    return data['data']

def get_submissions_from_reddit_api(submission_ids):
    '''
    Use the Reddit API to get most updated metadata, such as scores. Useful for most up to data text & scores, 
    but not necessary for data collection
    '''
    headers = {'User-agent':'Submission Collector'}
    params = {}
    params['id'] = ','.join(["t3_" + id for id in submission_ids])
    r = requests.get("https://api.reddit.com/api/info/", params=params, headers=headers)
    data = r.json()
    return data['data']['children']

def in_il(text):
    lower_text = text.lower()
    within_il = True in list(map(lambda x: x in lower_text, il_counties)) or \
        True in list(map(lambda x: x in lower_text, il_cities))

    within_other = True in list(map(lambda x: x in lower_text, other_states)) or \
        True in list(map(lambda x: x in lower_text, other_cities)) or \
        True in list(map(lambda x: x in lower_text, other_regions)) or \
        True in list(map(lambda x: x in lower_text, other_countries)) or \
        True in list(map(lambda x: x in lower_text, other_states_abbrev)) or \
        True in list(map(lambda x: x in lower_text, other_states_abbrev_1)) or \
        True in list(map(lambda x: x in lower_text, other_states_abbrev_2)) or \
        True in list(map(lambda x: x in lower_text, other_states_abbrev_3))
    return within_il and not within_other
    
def filter_il(check_il, submission_list):
    ret = []
    for entry in submission_list:
        entry_text = entry['selftext'] if 'selftext' in entry else ''
        whole_text = entry['title'] + ' ' + entry_text
        if '[deleted]' in whole_text or '[removed]' in whole_text or len(entry_text) < 5:
            continue
        if not check_il or in_il(whole_text):
            ret.append(entry)
    return ret

def trim_pushshift_data(check_il, submissions):
    '''
    parameters: list of submission details from Pushshift API
    returns: list of submission details, but with only wanted attributes to clean and limit data size
    '''
    relevant_attributes = {'title', 'created_utc', 'full_link', 'score', 'id', 'selftext', 'is_self'}
    return filter_il(check_il, [{k: v for k, v in submission.items() if k in relevant_attributes} for submission in submissions])

def update_data_with_RedditAPI(submissions):
    '''
    Scores retrieved from Pushshift may not but up to date. Use Reddit API to get most recent updated data.
    '''
#     submission_ids = [submission['id'] for submission in submissions]
#     get_submissions_from_reddit_api(submission_ids)
    for submission in submissions:
#         print(submission)
#         print(get_submissions_from_reddit_api([submission['id']]))
        try:
            r = requests.get("https://api.reddit.com/api/info/", params={'id':'t3_' + submission['id']}, headers={'User-agent':'Submission Collector'})
            data = r.json()['data']['children']
            submission['score'] = data[0]['data']['score']    
        except:
            pass
#             print(r)
#         submission['score'] = get_submissions_from_reddit_api([submission['id']])[0]['data']['score']    
#         print(submission['score'])

def get_all_submissions_from_subreddit(check_il, subreddit, after=None, before=None, score=None):
    allSubredditData = [] # Collect all data
    # Loop in batches
    start_time = time.time()
    while True:
        start_batch = time.time()
        # This will get the submission ids from Pushshift in batches of 1000 -- Reddit's API only allows 100 at a time
        submissions = get_submissions_from_pushshift(subreddit=subreddit, size=1000, after=after, before=before, score=score)
        if not submissions: 
            break
        # Trim data retrieved from Pushshift into managable dictionaries.
        submissions = trim_pushshift_data(check_il, submissions)
        # submissions = update_data_with_RedditAPI(submissions)
        allSubredditData.extend(submissions)
        if len(submissions) > 0:
            after = submissions[-1]['created_utc']
            end_batch = time.time()
            print('Last Date:', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(after)))
            print("Retrieved {} results in {:.4f}s. Total results so far: {}".format(len(submissions), end_batch - start_batch, len(allSubredditData)))
        else:
            break
    
    end_time = time.time()
    print("Retrieved {} total results in {:.4f}s. ".format(len(allSubredditData), end_time - start_time))
    return allSubredditData

In [8]:
queries = [{'before': None, 'after': int(datetime.datetime(year=2019, month=12, day=1).timestamp()), 'subreddit': "coronavirus"},
           {'before': None, 'after': int(datetime.datetime(year=2019, month=12, day=1).timestamp()), 'subreddit': "legaladvice"},
           {'before': None, 'after': int(datetime.datetime(year=2019, month=12, day=1).timestamp()), 'subreddit': "Chicago"},
           {'before': None, 'after': int(datetime.datetime(year=2019, month=12, day=1).timestamp()), 'subreddit': "ChicagoSuburbs"},
           {'before': None, 'after': int(datetime.datetime(year=2019, month=12, day=1).timestamp()), 'subreddit': "illinois"},
           {'before': None, 'after': int(datetime.datetime(year=2019, month=12, day=1).timestamp()), 'subreddit': "Northwestern"},
           {'before': None, 'after': int(datetime.datetime(year=2019, month=12, day=1).timestamp()), 'subreddit': "uchicago"},
           {'before': None, 'after': int(datetime.datetime(year=2019, month=12, day=1).timestamp()), 'subreddit': "UIUC"},
           {'before': None, 'after': int(datetime.datetime(year=2019, month=12, day=1).timestamp()), 'subreddit': "COVID19"},
           {'before': None, 'after': int(datetime.datetime(year=2019, month=12, day=1).timestamp()), 'subreddit': "news"},
           {'before': None, 'after': int(datetime.datetime(year=2019, month=12, day=1).timestamp()), 'subreddit': "law"},
           {'before': None, 'after': int(datetime.datetime(year=2019, month=12, day=1).timestamp()), 'subreddit': "LawSchool"}]

check_il = {'coronavirus': True,
            'legaladvice': True,
            'Chicago': False,
            'illinois': False,
            'ChicagoSuburbs': False,
            'Northwestern': False,
            'uchicago': False,
            'UIUC': False,
            'COVID19': True,
            'law': True,
            'news': True,
            'LawSchool': True
}
for query in queries:
    data = get_all_submissions_from_subreddit(check_il[query['subreddit']], 
        query['subreddit'], before=query['before'], after=query['after'])
    with open(query['subreddit'] + '.json', 'w') as f:
        json.dump(data, f)

Last Date: 2020-01-26 10:12:22
Retrieved 20 results in 2.9663s. Total results so far: 20
Last Date: 2020-01-27 21:49:38
Retrieved 21 results in 3.5165s. Total results so far: 41
Last Date: 2020-01-29 05:42:16
Retrieved 19 results in 2.9734s. Total results so far: 60
Last Date: 2020-01-30 13:11:43
Retrieved 13 results in 3.0690s. Total results so far: 73
Last Date: 2020-01-31 14:37:49
Retrieved 12 results in 4.7157s. Total results so far: 85
Last Date: 2020-02-01 21:57:26
Retrieved 13 results in 4.4898s. Total results so far: 98
Last Date: 2020-02-03 07:07:29
Retrieved 14 results in 2.6392s. Total results so far: 112
Last Date: 2020-02-04 15:23:34
Retrieved 15 results in 2.8913s. Total results so far: 127
Last Date: 2020-02-05 22:42:36
Retrieved 22 results in 6.7470s. Total results so far: 149
Last Date: 2020-02-07 01:39:47
Retrieved 15 results in 4.1532s. Total results so far: 164
Last Date: 2020-02-08 09:58:11
Retrieved 8 results in 4.4810s. Total results so far: 172
Last Date: 2020-0

Last Date: 2020-02-13 05:15:15
Retrieved 7 results in 8.2502s. Total results so far: 404
Last Date: 2020-02-14 08:20:13
Retrieved 9 results in 8.6251s. Total results so far: 413
Last Date: 2020-02-15 17:45:16
Retrieved 7 results in 8.3725s. Total results so far: 420
Last Date: 2020-02-17 07:21:01
Retrieved 9 results in 8.3292s. Total results so far: 429
Last Date: 2020-02-18 11:09:59
Retrieved 4 results in 8.3518s. Total results so far: 433
Last Date: 2020-02-19 13:54:13
Retrieved 5 results in 5.5980s. Total results so far: 438
Last Date: 2020-02-20 18:51:41
Retrieved 6 results in 5.0686s. Total results so far: 444
Last Date: 2020-02-21 20:46:05
Retrieved 5 results in 5.6505s. Total results so far: 449
Last Date: 2020-02-23 13:44:23
Retrieved 11 results in 4.8236s. Total results so far: 460
Last Date: 2020-02-24 10:58:03
Retrieved 6 results in 5.2833s. Total results so far: 466
Last Date: 2020-02-25 14:10:50
Retrieved 9 results in 9.3488s. Total results so far: 475
Last Date: 2020-02-2

Retrieved 144 total results in 37.4720s. 
